# build a model
- we can use sklearn model directly
- can also add preprocessing or feature extraction algorithms here
  - preprocessing : http://scikit-learn.org/stable/modules/preprocessing.html
  - extraction : http://scikit-learn.org/stable/modules/feature_extraction.html
  - integrate with pipeline: http://scikit-learn.org/stable/modules/pipeline.html
- select parameter:
  - cross validation : http://scikit-learn.org/stable/modules/cross_validation.html
  - grid search the parameter : http://scikit-learn.org/stable/modules/grid_search.html#exhaustive-grid-search

In [ ]:
%%writefile ./competition/models/impute_rf.py
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier

estimators = [('impute', Imputer(missing_values='NaN', strategy='most_frequent', axis=0)),
              ('clf', RandomForestClassifier())]

estimator = Pipeline(estimators)

tuned_parameters = [{
        'impute__copy':[False],
        'clf__n_estimators': [10],
        'clf__min_samples_split':[2],
        'clf__min_samples_leaf':[1],
        'clf__min_weight_fraction_leaf':[0],
        'clf__max_leaf_nodes':[None],
        'clf__min_impurity_split':[1e-7],
        'clf__warm_start':[False],
        'clf__oob_score':[False],
        'clf__class_weight':['balanced']
},]


In [ ]:
# modify config.py
!cat config.py

## tuning parameters

In [ ]:
%run select_models.py

# predict result

In [ ]:
%run save_result.py

# view result

In [ ]:
from config import extractor_name,estimator_name
import pandas as pd
destdir = './_results/%s-%s/'%(extractor_name,estimator_name)
sub = pd.read_csv(destdir+'submission.csv')

In [ ]:
sub.describe()

In [ ]:
sub

In [ ]:
from data import store
test = store['test']
test.shape